# Celestial coordinates

The [astropy.coordinates](https://docs.astropy.org/en/stable/coordinates/index.html) sub-package in astropy provides a way to represent and manipulate celestial coordinates. It supports many different coordinate systems and transformations, and includes support for velocities in addition to positions.


<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span> Objectives</h2>
</div>


<div class="panel-body">

<ul>
<li>Create SkyCoord objects</li>
<li>Access individual coordinates</li>
<li>Format coordinates</li>
<li>Transforming coordinates</li>
<li>Using 3D coordinates and velocities</li>
<li>Calculating separations</li>
<li>Using arrays of coordinates</li>
</ul>

</div>

</section>


## Documentation

This notebook only shows a subset of the functionality in astropy.coordinates. For more information about the features presented below as well as other available features, you can read the
[astropy.coordinates documentation](https://docs.astropy.org/en/stable/coordinates/).

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('image', origin='lower')
plt.rc('figure', figsize=(10, 6))

## Creating SkyCoord objects

The main user-facing class in astropy.coordinates is the ``SkyCoord`` class:

In [2]:
from astropy.coordinates import SkyCoord

A SkyCoord object can represent one or more coordinates, and there are a number of different ways to initialize such an object. If you have the coordinates as quantities, you can do:

In [3]:
from astropy import units as u
c1 =  SkyCoord(13.44 * u.deg, -33.298 * u.deg)
c1

<SkyCoord (ICRS): (ra, dec) in deg
    (13.44, -33.298)>

As you can see, the default is to assume that the coordinates are in the ICRS frame and represent Right Ascension and Declination. You can also initialize coordinates from strings:

In [4]:
c2 = SkyCoord("13:18:22", "-21:43:34", unit=(u.hourangle, u.deg))
c2

<SkyCoord (ICRS): (ra, dec) in deg
    (199.59166667, -21.72611111)>

In [7]:
c3 = SkyCoord("13h18m22s", "-21d43m34s")
c3

<SkyCoord (ICRS): (ra, dec) in deg
    (199.59166667, -21.72611111)>

To create coordinates in other frames - for example Galactic coordinates - you can use the ``frame=`` argument:

In [9]:
c4 = SkyCoord(l=13.4*u.deg, b=0.1*u.deg, frame="galactic")
c4

<SkyCoord (Galactic): (l, b) in deg
    (13.4, 0.1)>

Note that here we've also specified the names of the coordinates explicitly using ``l=`` and ``b=`` (they are not required but can make code more readable).

## Initialization from SIMBAD identifier

If you are doing quick calculations and want to use the coordinates of a source in the SIMBAD database, you can easily create a ``SkyCoord`` object from the object name:

In [11]:
m42 = SkyCoord.from_name("M42")
m42

<SkyCoord (ICRS): (ra, dec) in deg
    (83.82208333, -5.39111111)>

However, note that this should not be used when precision is critical - the positions are approximate especially for extended sources.

## Accessing individual coordinates

Once you have a coordinate object, you can access the individual coordinates using attributes:

In [14]:
c3.ra

<Longitude 199.59166667 deg>

In [15]:
type(c3.ra)

astropy.coordinates.angles.Longitude

The Longitude class is a sub-class of Quantity so you can convert it to specific units as you would a quantity:

In [22]:
c3.ra.to(u.hourangle)

<Longitude 13.30611111 hourangle>

but the Longitude class also provides shortcuts for this:

## Formatting coordinates

You can format a coordinate object as a string using ``to_string()``:

In [24]:
c3.to_string()

'199.592 -21.7261'

The default format uses floating point values, but you may want instead to use standard sexagesimal format: 

In [26]:
c3.to_string("hmsdms")

'13h18m22s -21d43m34s'

You can control the precision using the ``precision=`` keyword argument:

In [28]:
c3.to_string("hmsdms", precision=3)

'13h18m22.000s -21d43m34.000s'

## Transforming coordinates

``SkyCoord`` objects can be transformed between frames using the ``.transform_to()`` method:

In [29]:
c3.transform_to("galactic")

<SkyCoord (Galactic): (l, b) in deg
    (311.19233578, 40.71332142)>

You can also specify frame objects if you need more control:

In [30]:
from astropy.coordinates import FK5
c3.transform_to(FK5(equinox="J2020"))

<SkyCoord (FK5: equinox=J2020.000): (ra, dec) in deg
    (199.86293798, -21.83091578)>

A more advanced example is to convert the coordinates to altitude/azimuth as seen from a particular observer at a specific time (in UTC):

In [49]:
from astropy.coordinates import EarthLocation, AltAz
from astropy.time import Time

In [50]:
seattle = EarthLocation(lat=47.6062 * u.deg, lon=-122.3321 * u.deg, height= 0*u.m)
utc_time = Time.now()

In [51]:
coord_altaz = m42.transform_to(AltAz(obstime=utc_time, location=seattle))

In [52]:
coord_altaz

<SkyCoord (AltAz: obstime=2021-01-08 22:07:23.122096, location=(-2304112.88992931, -3640229.22416271, 4687466.69702659) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (222.29161396, 14.77501139)>

In [53]:
coord_altaz.alt

<Latitude 14.77501139 deg>

## 3D coordinates

The above examples are for coordinates on the sky, which are assumed to be at infinity. However, it is also possible to specify a third coordinate (the distance to the object):

In [54]:
crab = SkyCoord("05h34m 22d00m", distance=2*u.kpc)
crab

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    (83.5, 22., 2.)>

We can convert this to the Galactocentric frame, which is a frame that represents coordinates in cartesian coordinates by default (centered on the center of the Milky Way).

In [55]:
crab_galcen = crab.transform_to("galactocentric")
crab_galcen

<SkyCoord (Galactocentric: galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg): (x, y, z) in kpc
    (-10.10576999, -0.15620074, -0.17958032)>

In [56]:
crab_galcen.x

<Quantity -10.10576999 kpc>

In [58]:
crab_galcen.y

<Quantity -0.15620074 kpc>

In [60]:
crab_galcen.z

<Quantity -0.17958032 kpc>

## Velocities/proper motions

In addition to being able to represent fixed positions in space, it is also possible to add velocities to coordinates, for example to express radial velocity, proper motions, or 3D space motion. For example, to add a radial velocity and proper motion to a coordinate in the ICRS frame, you can do:

In [61]:
coord4 = SkyCoord(ra=13.443 * u.deg,
                  dec=-33.298 * u.deg, distance=1 * u.kpc,
                  pm_ra_cosdec=0.1 * u.arcsec / u.yr,
                  pm_dec=-0.08 * u.arcsec / u.yr,
                  radial_velocity=20 * u.km / u.s)
coord4

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    (13.443, -33.298, 1.)
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (100., -80., 20.)>

Velocities are then correctly propagated when transforming to different systems:

In [62]:
coord4.transform_to("galactic")

<SkyCoord (Galactic): (l, b, distance) in (deg, deg, kpc)
    (298.40424611, -83.80972028, 1.)
 (pm_l_cosb, pm_b, radial_velocity) in (mas / yr, mas / yr, km / s)
    (-92.92161928, 88.12248675, 20.)>

In [64]:
coord4.transform_to("galactocentric")

<SkyCoord (Galactocentric: galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg): (x, y, z) in kpc
    (-8.07322429, -0.0948479, -0.97349744)
 (v_x, v_y, v_z) in km / s
    (-175.91563317, -331.14327184, 33.4245431)>

## Separations

Given two coordinates, we can determine the separation between them on the sky:

In [65]:
c1.separation(c3)

<Angle 124.66384628 deg>

We can also do this in 3D to find a physical distance:

In [68]:
coord4.separation_3d(crab_galcen)

<Distance 2.18295914 kpc>

## Using arrays in coordinates

A common use case is the need to represent large catalogs of coordinates - in this case, it is very inefficient to use a ``SkyCoord`` object for each source. Instead, you can store arrays of coordinates inside ``SkyCoord``:

In [69]:
coord5 = SkyCoord(ra=[1, 2, 3]*u.deg, dec=[4,5,6]*u.deg)
coord5

<SkyCoord (ICRS): (ra, dec) in deg
    [(1., 4.), (2., 5.), (3., 6.)]>

Most of the operations shown above should then also work seamlessly with arrays:

In [71]:
coord5.separation(c1).to(u.deg)

<Angle [39.11157274, 39.80248312, 40.52688133] deg>

In [73]:
coord6 = SkyCoord(ra=[2, 3, 1, 7, 10, ]*u.deg, dec=[5, 6, 4, 14, 15]*u.deg) 
coord5.match_to_catalog_sky(coord6)

(array([2, 0, 1]), <Angle [0., 0., 0.] deg>, <Quantity [0., 0., 0.]>)

## Solar system bodies

A [get_body](https://docs.astropy.org/en/stable/api/astropy.coordinates.get_body.html) convenience function is provided in astropy.coordinates to retrieve the 3D coordinates of solar system bodies. This can be done either using default built-in ephemeris, or the more accurate JPL ephemeris (see the ``get_body`` documentation). For now, we will use the default:

In [75]:
from astropy.time import Time
from astropy.coordinates import get_body

jupiter = get_body("Jupiter", Time.now())

In [76]:
jupiter

<SkyCoord (GCRS: obstime=2021-01-08 22:22:02.354730, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, AU)
    (306.78111977, -19.66017349, 6.03230825)>


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<p>The answer to some of the following can be found in <a href="https://docs.astropy.org/en/stable/time/index.html">the documentation</a>!</p>
<ol>
<li>Find the coordinates of the Crab Nebula (M1) in ICRS coordinates, and convert them to Galactic Coordinates.</li>
<li>Find the separation on the sky between the Crab Nebula and the star 'T Tauri' in degrees</li>
<li>Generate random positions in the Galactic plane (between l=0 and l=360 and b=-1 and b=+1), then make a plot showing the position on the sky of these points in FK5.</li>
<li>Make a plot showing the altitude above the horizon of the Sun as seen from Madrid over the course of today</li>
</ol>

</div>

</section>


<center><i>This notebook was written by <a href="https://aperiosoftware.com/">Aperio Software Ltd.</a> &copy; 2019, and is licensed under a <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License (CC BY 4.0)</a></i></center>

![cc](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by.svg)